## Test et présentation du projet NFT

#### Importations

In [2]:
from scripts.NFT import Collection, NFT
from scripts.timestamp import now
import time

#### Créer une collection

In [3]:
ownerKey = "OwnerPublicKey_1001101"
userKey1 = "user1Key_1011001"
userKey2 = "user2Key_0100110"

# création d'une collection

collection = Collection(ownerKey, 4, 10) # remplis les champs : userPublicKey, limit et mintDuration

# autorisation de user1 ( en tant que owner)

collection.add_minter(userKey1, ownerKey)

# affichage de la collection et test

assert (userKey1 in collection.authorizedMinters) 

"Success!"


collection.display()


{'owner': 'OwnerPublicKey_1001101',
 'limit': 4,
 'mintingOpen': False,
 'mintStart': None,
 'NFTs': [],
 'blockchain': {'type': 'Blockchain',
  'blocks': [{'type': 'Block',
    'issuer': '......',
    'timestamp': 0,
    'hash': '93ae8c7bcd3208b5...',
    'signature': '...',
    'index': 0,
    'parent': '0000000000000000...',
    'certificates': [],
    'nonce': 0}]}}

#### Ouverture de la période de Minting
##### Test par utilisateur non propriétaire et utilisateur owner

In [4]:
# Utilisateur non autorisé user2 par exemple

collection.open_mint(userKey2)
# doit rendre une erreur


Exception: Pas autorisé à ouvrir le minting

In [5]:
# utilisateur autorisé (owner)

collection.open_mint(ownerKey)

assert collection.mintingOpen == True

'success'

'success'

##### Test depassement de la durée de minting

In [6]:
# obligation d'utiliser check_mint_period (pas de tache en arrière plan en synchrone)

collection.open_mint(ownerKey)
print(collection.mintingOpen)

time.sleep(11)

collection.check_mint_period()
print(collection.mintingOpen)



True
False


#### test minting

In [10]:
# Reset
collection = Collection(ownerKey, 4, 10)
collection.add_minter(userKey1, ownerKey)

# test par un utilisateur authorisé (owner)
collection.open_mint(ownerKey)
token_id1 = collection.mint(ownerKey)
assert token_id1 == 1 #Le premier token_id doit être 1
assert token_id1 in collection.nftRegistry #Le NFT doit être enregistré dans le registre
nft1 = collection.nftRegistry[token_id1]
print("Mint effectué par le propriétaire:", nft1.display())

# user1 mint
# ouvrir la période de mint
collection.open_mint(ownerKey)

token_id2 = collection.mint(userKey1)
assert token_id2 == 2, "Le second token_id doit être = 2"
assert token_id2 in collection.nftRegistry, "Le NFT doit être enregistré dans le registre"
nft2 = collection.nftRegistry[token_id2]
print("Mint effectué par l'utilisateur 1:", nft2.display())


# vérification que l'image n'existe plus dans le registre
assert nft1.metadata not in collection.availableImages
assert nft2.metadata not in collection.availableImages

'success'




Mint effectué par le propriétaire: {'token_id': 1, 'metadata': 'TrafalgarToad.jpg', 'owner': 'ublicKey_1001101'}
Mint effectué par l'utilisateur 1: {'token_id': 2, 'metadata': 'congratulations.webp', 'owner': 'user1Key_1011001'}


'success'

In [8]:
# test de mint par un utilisateur non autorisé (doit retourner une erreur)
#reset
collection = Collection(ownerKey, 4, 10)
collection.add_minter(userKey1, ownerKey)

collection.open_mint(ownerKey)
token_id1 = collection.mint(userKey2)


Exception: Utilisateur non autorisé à mint

#### Transfert de NFT

In [12]:
collection.open_mint(ownerKey)
token_id1 = collection.mint(ownerKey)
nft1 = collection.nftRegistry[token_id1]

# on test en transférant le NFT 1 du proprietaire à l'user2 

print(collection.nftRegistry)
result = collection.transfer(token_id1, ownerKey, userKey2)
assert result == True

print("Transfert du NFT 1 réussi. Nouvelle propriété:", collection.nftRegistry[token_id1].display())

{1: <scripts.NFT.NFT object at 0x0000029213A30980>, 2: <scripts.NFT.NFT object at 0x000002921378FFE0>, 3: <scripts.NFT.NFT object at 0x0000029213A30AD0>}
Transfert du NFT 1 réussi. Nouvelle propriété: {'token_id': 3, 'metadata': 'mickeyDog.jpg', 'owner': 'user2Key_0100110'}


In [13]:
# utilisateur 1 essaye de transférer un nft pas à lui
# doit retourner une erreur

print(collection.nftRegistry)
result = collection.transfer(token_id1, userKey1, userKey2)



{1: <scripts.NFT.NFT object at 0x0000029213A30980>, 2: <scripts.NFT.NFT object at 0x000002921378FFE0>, 3: <scripts.NFT.NFT object at 0x0000029213A30AD0>}


Exception: Error : Pas autorisé à transferer le NFT

### Test de méthode d'affichage

In [14]:
print(collection.nftRegistry)

info_collection = collection.display()
# doit contenir le propriétaire, la liste de NFT et la blockchain
assert 'owner' in info_collection
assert 'NFTs' in info_collection
assert 'blockchain' in info_collection
'success'

print("Affichage de la collection :", info_collection)

{1: <scripts.NFT.NFT object at 0x0000029213A30980>, 2: <scripts.NFT.NFT object at 0x000002921378FFE0>, 3: <scripts.NFT.NFT object at 0x0000029213A30AD0>}
Affichage de la collection : {'owner': 'OwnerPublicKey_1001101', 'limit': 4, 'mintingOpen': True, 'mintStart': 1741198978, 'NFTs': [{'token_id': 1, 'metadata': 'TrafalgarToad.jpg', 'owner': 'user2Key_0100110'}, {'token_id': 2, 'metadata': 'congratulations.webp', 'owner': 'user1Key_1011001'}, {'token_id': 3, 'metadata': 'mickeyDog.jpg', 'owner': 'user2Key_0100110'}], 'blockchain': {'type': 'Blockchain', 'blocks': [{'type': 'Block', 'issuer': '......', 'timestamp': 0, 'hash': '93ae8c7bcd3208b5...', 'signature': '...', 'index': 0, 'parent': '0000000000000000...', 'certificates': [], 'nonce': 0}, {'type': 'Block', 'issuer': '......', 'timestamp': 1741198940, 'hash': '0470a6830fafcf09...', 'signature': '...', 'index': 1, 'parent': '93ae8c7bcd3208b5...', 'certificates': [{'type': 'Certificate', 'issuer': '......', 'timestamp': 1741198940, '

## Scénario test
### un ami a créer une collection de nft et m'invite à le rejoindre pour mint ensemble ??? transferer des NFT... 

#### On commence par poser les bases, créer la collection du point de vue de Splinter (l'ami) 

In [15]:
#Mon ami Splinter crée sa clé
SplinterKey = "OwnerPublicKey_Mon_ami_Splinter"

# Ensuit il déclare une collection, vide pour l'instant, Splinter met 2 NFT comme limite et 5 secondes pour le mint
collection = Collection(SplinterKey, 2, 5)
collection.add_minter(SplinterKey, SplinterKey)

#### Splinter m'invite ensuite à mint 

In [16]:
MyKey = "OwnerPublicKey_Moi"

# n'ayant pas trop idée de quoi il s'agit j'essaye de mint directement un nft

collection.mint(MyKey)

Exception: Période de minting fermée

#### Mince la période de minting est fermée, j'essaye de l'ouvrir de mon coté

In [17]:
collection.open_mint(MyKey)

Exception: Pas autorisé à ouvrir le minting

#### Mince, je ne peux pas faire ça non plus autant appeler Splinter pour lui demander de m'accorder les droits d'accés

In [18]:
MyKey = "OwnerPublicKey_Moi"
print(collection.authorizedMinters)

# une conversation téléphonique plus tard

# Splinter m'ajoute en tant que minter
collection.add_minter(MyKey, SplinterKey)

# et comme il est prévoyant il ouvre la période de mint
collection.open_mint(SplinterKey)



{'OwnerPublicKey_Mon_ami_Splinter'}


#### Maintenant je peux mint mes premiers NFTs 

In [49]:
collection = Collection(SplinterKey, 2, 5)
collection.add_minter(MyKey, SplinterKey)

#En parallèle Splinter rouvre la période de mint
collection.open_mint(SplinterKey)

# je mint mes NFTs
token_id1 = collection.mint(MyKey)
nft1 = collection.nftRegistry[token_id1]
print("Mint effectué par le propriétaire:", nft1.display())

# trop bien ça marche j'en refais !
token_id2 = collection.mint(MyKey)
nft2 = collection.nftRegistry[token_id2]
print("Mint effectué par le propriétaire:", nft2.display())


Mint effectué par le propriétaire: {'token_id': 1, 'metadata': 'BlackLeg.jpg', 'owner': 'nerPublicKey_Moi'}
Mint effectué par le propriétaire: {'token_id': 2, 'metadata': 'GokuChauve.webp', 'owner': 'nerPublicKey_Moi'}


In [20]:
collection.open_mint(SplinterKey)

# J'aime beaicoup je vais en mint un autre
token_id3 = collection.mint(MyKey)
nft3 = collection.nftRegistry[token_id3]

Exception: plus de place dans la collection NFTs

#### Ah mince c'est vrai que la limite de NFTs est limité à 2 (classique Splinter !)
#### En parallèle Splinter essaye de mint un NFT mais erreur ! il décide donc d'aller voir la collection

In [33]:
collection.display()

{'owner': 'OwnerPublicKey_Mon_ami_Splinter',
 'limit': 2,
 'mintingOpen': True,
 'mintStart': 1741199526,
 'NFTs': [{'token_id': 1,
   'metadata': 'GokuChauve.webp',
   'owner': 'nerPublicKey_Moi'},
  {'token_id': 2, 'metadata': 'SonicLane.jpg', 'owner': 'nerPublicKey_Moi'}],
 'blockchain': {'type': 'Blockchain',
  'blocks': [{'type': 'Block',
    'issuer': '......',
    'timestamp': 0,
    'hash': '93ae8c7bcd3208b5...',
    'signature': '...',
    'index': 0,
    'parent': '0000000000000000...',
    'certificates': [],
    'nonce': 0},
   {'type': 'Block',
    'issuer': '......',
    'timestamp': 1741199526,
    'hash': 'bc6b98596dea0bdf...',
    'signature': '...',
    'index': 1,
    'parent': '93ae8c7bcd3208b5...',
    'certificates': [{'type': 'Certificate',
      'issuer': '......',
      'timestamp': 1741199526,
      'hash': 'd7a2327365dcd33c...',
      'signature': '...'}],
    'nonce': 0},
   {'type': 'Block',
    'issuer': '......',
    'timestamp': 1741199526,
    'hash': '

In [50]:
import os

# la limite est bien atteinte Splinter me demande de lui transférer un NFT, étant un bon ami j'accepte

collection.transfer(token_id1, MyKey, SplinterKey)

# Splinter vérifie que le transfert a bien été effectué
print("NFTs de la collection:", collection.nftRegistry)

collection.display()

# Parfait maintenant Splinter veut voir l'image de son NFT
open(os.path.join(collection.imagePath, nft1.metadata), 'rb').read()



NFTs de la collection: {1: <scripts.NFT.NFT object at 0x0000029214656C90>, 2: <scripts.NFT.NFT object at 0x000002921475D430>}


b'\xff\xd8\xff\xdb\x00\xc5\x00\x04\x05\x05\t\x06\t\t\t\t\t\n\x08\t\x08\n\x0b\x0b\n\n\x0b\x0b\x0c\n\x0b\n\x0b\n\x0c\x0c\x0c\x0c\r\r\x0c\x0c\x0c\x0c\x0c\x0f\x0e\x0f\x0c\x0c\r\x0f\x0f\x0f\x0f\r\x0e\x11\x11\x11\x0e\x11\x10\x10\x11\x13\x11\x13\x11\x11\r\x01\x04\x06\x06\n\t\n\x0b\n\n\x0b\x0b\x0c\x0c\x0c\x0b\x0f\x10\x12\x12\x10\x0f\x12\x10\x11\x11\x11\x10\x12\x1e"\x1c\x11\x11\x1c"\x1e\x17j\x1a\x13\x1aj\x17\x1a\x1f\x0f\x0f\x1f\x1a*\x11\x1f\x11*<..<\x0f\x0f\x0f\x0f\x0ft\x02\x04\x04\x04\x08\x06\x08\x07\x08\x08\x07\x08\x06\x08\x06\x08\x08\x08\x07\x07\x08\x08\t\x07\x07\x07\x07\x07\t\n\t\x08\x08\x08\x08\t\n\t\x08\x08\x06\x08\x08\t\t\t\n\n\t\t\n\x08\t\x08\n\n\n\n\n\x0e\x10\x0e\x0e\x0ew\xff\xc2\x00\x11\x08\x01\x12\x00\xec\x03\x01"\x00\x02\x11\x01\x03\x11\x02\xff\xc4\x00\x98\x00\x00\x00\x07\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x00\x08\t\x10\x00\x01\x04\x02\x02\x02\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x01\x00\x02\x03\x04\x05\x06\x10\x11\x07\x12\x13\x14 \x15